<a href="https://colab.research.google.com/github/tiagomiranda24/Lab.-Bioinformatic/blob/main/lab_bioinf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  aceder ao NCBI e guardar os ficheiros correspondentes aos genes escolhidos, podendo explorar possíveis variantes;


In [ ]:
pip install BioPython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.5 MB/s eta 0:00:00


In [2]:
import Bio
from Bio import Entrez

Entrez.email = 'pg52526@alunos.uminho.pt'

orfs = ['SALL3', 'RNF130', 'RIMS2']
id = ['Q62255', 'Q86XS8', 'Q9UQ26']
# SALL3 - aminoacid sequence                  Uniprot id = Q62255
# RNF130 - aminoacid sequence                 Uniprot id = Q86XS8
# RIMS2 - SARS-CoV-2 aminoacid sequence       Uniprot id = Q9UQ26

for n in range(len(orfs)):
    handle = Entrez.efetch(db='Protein', id=id[n], rettype='fasta', retmode='text')
    file = open(orfs[n] + '.fa', 'w')
    file.writelines(handle)
    file.close()
    handle.close()

    handle = Entrez.efetch(db='Protein', id=id[n], rettype='gb', retmode='text')
    file = open(orfs[n] + '.gb', 'w')
    file.writelines(handle)
    file.close()
    handle.close()

https://docs.google.com/document/d/1yMAcnL-2XoCSIjLQNpVvQhX0BkZdnSkqxLvka1x784k/edit

In [ ]:
# features e anotações

In [3]:
from Bio import SeqIO
orfs = ['SALL3', 'RNF130', 'RIMS2']

for orf in orfs:
    record = SeqIO.read(orf + ".gb", "genbank")
    print('ID:', record.id)
    print('Nome:', record.name)
    print('Descrição:', record.description)
    print(f'Tamanho da sequência:{len(record.seq)} aa')
    print('Referências a bases de dados (externas):', record.dbxrefs)
    print('Número de anotações:', len(record.annotations))
    print('Fonte de anotação:', record.annotations["source"])

    print('Número de features', len(record.features))
    print('Taxonomia:', *record.annotations['taxonomy'])
    print('Organismo:', record.annotations['organism'])
    print('Comentário:')
    print(record.annotations['comment'])

    featregion = [i for i in range(len(record.features)) if record.features[i].type == 'Region']
    print()
    print('Features:')
    print()

    for k in featregion:
        print(*record.features[k].qualifiers['region_name'])
        print(record.features[k].location)
        print(record.features[k].extract(record.seq))
        print(record.features[k].qualifiers['note'])
        if k != featregion[-1]:
            print('---------')
    print()
    if orf != orfs[-1]:
        print('############################################')
    print()

ID: Q62255.2
Nome: SALL3_MOUSE
Descrição: RecName: Full=Sal-like protein 3; AltName: Full=MSal; AltName: Full=Spalt-like protein 3
Tamanho da sequência:1320 aa
Referências a bases de dados (externas): []
Número de anotações: 13
Fonte de anotação: Mus musculus (house mouse)
Número de features 61
Taxonomia: Eukaryota Metazoa Chordata Craniata Vertebrata Euteleostomi Mammalia Eutheria Euarchontoglires Glires Rodentia Myomorpha Muroidea Muridae Murinae Mus Mus
Organismo: Mus musculus
Comentário:
On or before Jan 23, 2007 this sequence version replaced
gi:123778174, gi:81909072, gi:81884764, gi:81913523, gi:20455242.
[FUNCTION] Probable transcription factor.
[SUBCELLULAR LOCATION] Nucleus {ECO:0000305}.
[ALTERNATIVE PRODUCTS] Event=Alternative splicing; Named
isoforms=2; Name=1; IsoId=Q62255-1; Sequence=Displayed; Name=2;
IsoId=Q62255-2; Sequence=VSP_006834.
[TISSUE SPECIFICITY] In adult brain, testis and kidney. In lower
levels also in adult ovaries and embryonic stem cells. In embryo in
d

In [ ]:
#Blast

In [4]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import SearchIO



def get_prot(id):
    '''
    VARIAVEIS:
        id= codigo da proteina a pesquisar na base de dados Swiss-Prot;
    RETURNS:
        Lista de Acession numbers filtrados pelo tipo gb e sem repetições
    '''
    from Bio import ExPASy
    with ExPASy.get_sprot_raw(id) as handle:
        seq_record = SeqIO.read(handle, "swiss")
    print(seq_record.seq)
    print(len(seq_record.seq), 'aa')
    for k, v in seq_record.annotations.items():
        print(k, v)
    tam= len(seq_record.seq)
    seq= seq_record.seq
    tax= seq_record.annotations["taxonomy"]
    org= seq_record.annotations["organism"]
    #host= seq_record.annotations["organism_host"]
    y = ('ID:' + id + '|' + 'SEQUENCE:' + seq + '|' + 'SEQUENCE LENGTH:' + str(tam) + 'bp' + '|' + 'TAXONOMY:' + str(tax) + '|' + 'ORGANISM:' + org )
    return y

def filtro(seq):
    '''
    VARIAVEIS:
        seq = string resultado do get_prot(id)
    RETURNS:
        Lista da string separada pelas "|"
    '''
    seq = seq.split('|')
    return seq

def blast_prot(FILE, seq):
    '''
    VARIAVEIS:
        FILE = Nome do ficheiro a dar com o resultado do Blast ( ex: "FGB_blast.xml")
        seq - sequencia proteica query para executar o blast
    RETURNS:
        Devolve uma lista dos resultados do blast filtrados pelo e-value, com o nome do alinhamento, comprimento
        do alinhamento e e-value
    '''
    result_handle = NCBIWWW.qblast('blastp', 'nr', seq)
    with open(FILE, "w") as out_handle:
        out_handle.write(result_handle.read())
    result_handle.close()

def obter_homologos(file, E_VALUE_THRESH):
    '''
    VARIAVEIS:
        file = Nome do ficheiro a dar com o resultado do Blast ( ex: "FGB_blast.xml")
        E_VALUE_THRESH = Recebe valor None ou qualquer numero inteiro, corresponde ao valor de e-value maximo
        aceitável para tratamento do output do blast.
    RETURNS:
        Devolve uma lista dos resultados do blast filtrados pelo e-value, com o nome do alinhamento, comprimento
        do alinhamento e e-value
    '''
    result_handle = open(file)
    blast_record = NCBIXML.read(result_handle)
    FILE = str('SeqsHomologas_' + str(file) + '.fasta')
    ficheiro_output = open(FILE, 'w+')
    if E_VALUE_THRESH == None:
        E_VALUE_THRESH = 0.05
    for alignment in blast_record.alignments:
        for hsp in range(len(alignment.hsps)):
            if alignment.hsps[hsp].expect < E_VALUE_THRESH:
                if hsp != 0:
                    pass
                else:
                    ficheiro_output.write('>' + alignment.title + '\n' + alignment.hsps[hsp].sbjct + '\n')
    result_handle.close()

def parse_prot(file):
    '''
    VARIAVEIS:
        file = Nome do ficheiro resultado do Blast ( ex: "FGB_blast.xml")
    RETURNS:
        Informações contidas no ficheiro impressos sobre na consola.
    '''
    result_handle = open(file)
    from Bio.Blast import NCBIXML
    blast_record = NCBIXML.read(result_handle)
    E_VALUE_THRESH = 0.04
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            if hsp.expect < E_VALUE_THRESH:
                print("****Alignment****")
                print("sequence:", alignment.title)
                print("length:", alignment.length)
                print("e value:", hsp.expect)
                print(hsp.query[0:75] + "...")
                print(hsp.match[0:75] + "...")
                print(hsp.sbjct[0:75] + "...")
    from Bio import SearchIO
    blast_qresult = SearchIO.read(file, "blast-xml")
    print(blast_qresult)
    result_handle.close()


def proteico(id,file,blast = False, E_VALUE_THRESH = None):
    '''
    VARIAVEIS:
        genbank = ficheiro genbank com a extensão ponto gb (ex: prot.gb)
        id = id da base de SWISS Prot da proteina a ser tratada
        file = Nome do ficheiro a dar com o resultado do Blast ( ex: "FGB_blast.xml")
        blast = Em caso de omissão recebe valor  de False, se Blast = True irá utilizar o ficheiro genbank e
        realizar um blast.
        E_VALUE_THRESH = Recebe valor None ou qualquer numero inteiro, corresponde ao valor de e-value maximo
        aceitavel para tratamento do output do blast.
    RETURNS:
        Gera um ficheiro .txt contendo os Acession Number dos resultados do blast com filtração segundo o
        E_VALUE_THRESH e "gb".
        '''
    if blast == True:
        from Bio import ExPASy
        with ExPASy.get_sprot_raw(id) as handle:
            seq_record = SeqIO.read(handle, "swiss")
        blast_prot(file, seq_record.seq)
    x = get_prot(id)
    print(x)
    obter_homologos(file, E_VALUE_THRESH)
    parse_prot(file)

#proteico('P02671', 'FGG_prot_blast.xml', False, 0.05)

555555555555555555555555555555555555555555555555555555555555
# Multiple Alignment e Filogenia

from Bio import AlignIO
from Bio import Phylo

orfs = ['TRAF3', 'MAVS', 'ORF9b']

# ficheiros '..._alignment.clustal_num' e '..._tree.dnd' foram obtidos através de Clustal Omega online!

for orf in orfs:
    align = AlignIO.read(orf + '_alignment.clustal_num', 'clustal')
    print(align)

    tree = Phylo.read(orf + '_tree.dnd', 'newick')
    Phylo.draw_ascii(tree)


In [ ]:
# Multiple Alignment e Filogenia

from Bio import AlignIO
from Bio import Phylo

orfs = ['SALL3', 'RNF130', 'RIMS2']

# ficheiros '..._alignment.clustal_num' e '..._tree.dnd' foram obtidos através de Clustal Omega online!

for orf in orfs:
    align = AlignIO.read(orf + '_alignment.clustal_num', 'clustal')
    print(align)

    tree = Phylo.read(orf + '_tree.dnd', 'newick')
    Phylo.draw_ascii(tree)


In [ ]:
from Bio.Blast import NCBIXML, NCBIWWW
from Bio import SeqIO, AlignIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.Align import MultipleSeqAlignment, AlignInfo

Alignment example:

In [5]:
from Bio import Align

In [12]:
aligner = Align.PairwiseAligner(match_score = 1.0)

seq1 = "GAACT"
seq2 = "GAT"

# The score refers to the total number of base alignments that occurred
score = aligner.score(seq1, seq2)

print(f"The score of the alignment is {score}")

alignments = aligner.align(seq1, seq2)
for alignment in alignments:
    print(alignment)

aligner.mode = "local"

seq3 = "AGAACTC"
seq4 = "GAACT"

scoreLocal = aligner.score(seq3, seq4)
print(f"The local alignment score is {scoreLocal}")

alignLocal = aligner.align(seq3, seq4)
for x in alignLocal:
    print(x)

The score of the alignment is 3.0
target            0 GAACT 5
                  0 ||--| 5
query             0 GA--T 3

target            0 GAACT 5
                  0 |-|-| 5
query             0 G-A-T 3

The local alignment score is 5.0
target            1 GAACT 6
                  0 ||||| 5
query             0 GAACT 5

